In [93]:
import pandas as pd
import re

files = !ls data/debates/*.csv

full = pd.concat([pd.read_csv(f) for f in files])

In [94]:
# constants
title_pat=r'(.*)\((.*)\)'

In [95]:
# find patterns like PRESIDENTE DEL GOBIERNO (Mariano Rajoy)

# extract title, ex: "PRESDIENTE DEL GOBIERNO"
full["Title"]=full["Name"].apply(lambda x: re.match(title_pat, x).groups()[0] if re.match(title_pat, x)!=None else None)
# extract name, ex: "Mariano Rajoy"
full["Name"]=full["Name"].apply(lambda x: re.match(title_pat, x).groups()[1].upper() if re.match(title_pat, x)!=None else x)
full[full.Title.notnull()].head()

,Name,Text,Date,Term,Title
37,BELOKI GUERRA,Propuesta de transacción. Extender las relacio...,20080527,IX,SECRETARIO
89,SOLBES MIRA,"Señor presidente, señorías, comparezco de nuev...",20080626,IX,VICEPRESIDENTE SEGUNDO DEL GOBIERNO Y MINISTRO...
102,CUNILLERA I MESTRES,"Muchas gracias, señor Azpiazu. Tiene la palabr...",20080626,IX,VICEPRESIDENTA
104,CUNILLERA I MESTRES,"Señor Sánchez i Llibre, por favor",20080626,IX,VICEPRESIDENTA
106,CUNILLERA I MESTRES,"Gracias a usted, señor Sánchez. Tiene ahora la...",20080626,IX,VICEPRESIDENTA


In [96]:
# find paranthesized values and sort them by occurence
parenthesized_pat=r'.*(\(.+?\))'
# can also use negative matching instead of non-greedy parenthesized_pat=r'.*(\([^\)]+\))'
l=full["Text"].apply(lambda t: re.search(parenthesized_pat,str(t)).groups() if re.match(parenthesized_pat,str(t))!=None else None).dropna()
l=[item for tup in l for item in tup]
pd.DataFrame({'test':l}).groupby(['test']).size().reset_index().sort_values(0,ascending=False).head(20)

,test,0
9606,(Pausa.),24913
641,(Aplausos),24235
9598,(Pausa),10018
1638,(Convergència i Unió),6260
10520,(Rumores.),5187
9682,(Pausa.-Una trabajadora del servicio de limpie...,4616
10509,(Rumores),4054
662,(Aplausos.),3200
1720,(EAJ-PNV),2883
9510,(PNV),1158


In [97]:
# now remove parenthesized sentences
parenthesized_pat=r'\(.+?\)' # replace does not need groups like in the example above, will replace all the found regex
full["Text"]=full["Text"].str.replace(parenthesized_pat,'',regex=True)
full.head()


,Name,Text,Date,Term,Title
0,PRESIDENTE,"Señorías, se inicia la sesión. Tenemos como as...",20080527,IX,None
1,SECO REVILLA,"Sí, prometo. (Aplausos.",20080527,IX,None
2,PRESIDENTE,Don Óscar Seco Revilla ha adquirido la condici...,20080527,IX,None
3,PRESIDENTE,Pasamos a examinar el punto I del orden del dí...,20080527,IX,None
4,LLAMAZARES TRIGO,"Gracias, señor presidente. Señorías, con volun...",20080527,IX,None


In [98]:
full.to_parquet('../debates.parquet', index=False)

In [99]:
full[full['Name'].str.contains('EZ ZAPATERO')].to_csv('../zapatero.csv', index=False)
full[full['Name'].str.contains('RAJOY')].to_csv('../rajoy.csv',index=False)

In [100]:
full.to_csv('../debates.csv', index=False)